# Navigation

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the first project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893).

### 1. Start the Environment

We begin by importing some necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Banana.app"`
- **Windows** (x86): `"path/to/Banana_Windows_x86/Banana.exe"`
- **Windows** (x86_64): `"path/to/Banana_Windows_x86_64/Banana.exe"`
- **Linux** (x86): `"path/to/Banana_Linux/Banana.x86"`
- **Linux** (x86_64): `"path/to/Banana_Linux/Banana.x86_64"`
- **Linux** (x86, headless): `"path/to/Banana_Linux_NoVis/Banana.x86"`
- **Linux** (x86_64, headless): `"path/to/Banana_Linux_NoVis/Banana.x86_64"`

For instance, if you are using a Mac, then you downloaded `Banana.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Banana.app")
```

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [1]:
from unityagents import UnityEnvironment
import numpy as np
if False:
    env = UnityEnvironment(file_name="./Banana.app")
    # get the default brain
    brain_name = env.brain_names[0]
    brain      = env.brains[brain_name]

### 2. Examine the State and Action Spaces

The simulation contains a single agent that navigates a large environment.  At each time step, it has four actions at its disposal:
- `0` - walk forward 
- `1` - walk backward
- `2` - turn left
- `3` - turn right

The state space has `37` dimensions and contains the agent's velocity, along with ray-based perception of objects around agent's forward direction.  A reward of `+1` is provided for collecting a yellow banana, and a reward of `-1` is provided for collecting a blue banana. 

Run the code cell below to print some information about the environment.

In [2]:
if False:
    # reset the environment
    env_info = env.reset(train_mode=True)[brain_name]

    # number of agents in the environment
    print('Number of agents:', len(env_info.agents))

    # number of actions
    action_size = brain.vector_action_space_size
    print('Number of actions:', action_size)

    # examine the state space 
    state = env_info.vector_observations[0]
    print('States look like:', state)
    state_size = len(state)
    print('States have length:', state_size)

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action (uniformly) at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [3]:
do_sample = False
if do_sample:
    env_info = env.reset(train_mode=False)[brain_name] # reset the environment
    state = env_info.vector_observations[0]            # get the current state
    score = 0                                          # initialize the score
    _ = -1
    while True:
        _ += 1
        action     = np.random.randint(action_size)    # select an action
        env_info   = env.step(action)[brain_name]      # send the action to the environment
        next_state = env_info.vector_observations[0]   # get the next state
        reward     = env_info.rewards[0]               # get the reward
        done       = env_info.local_done[0]            # see if episode has finished
        score     += reward                            # update the score
        state      = next_state                        # roll over the state to next time step
        print(_, end=" ")
        if done:                                       # exit loop if episode finished
            break
    print("Score: {}".format(score))

When finished, you can close the environment.

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [4]:
def get_init(nb_action_unique):
    """
    Create 3 dict with auto creation of values zeros 
    """
    from collections import defaultdict
    nb_actions = nb_action_unique
    zeros      = lambda: np.zeros(nb_actions)
    N          = defaultdict(zeros) # compte le nombre de passage sur l'action (N = nombre de passage)
    R          = defaultdict(zeros) # somme les gains par case (R = return)
    Q          = defaultdict(zeros) # Moyenne du gain par nombre de passage (Q = euh...)
    return N,R,Q

In [ ]:
def print_info(i_episode,num_episodes, modulo=1000):
    import sys
    if i_episode % modulo == 0:
        print("\rEpisode {}/{}.".format(i_episode, num_episodes), end="")
        sys.stdout.flush()

In [ ]:
def generate_episode(env, brain_name):
    episode  = []
    env_info = env.reset(train_mode=False)[brain_name] # reset the environment
    state    = env_info.vector_observations[0]            # get the current state
    score    = 0                                          # initialize the score
    _ = -1
    length_episode = 50 # default = 300 ! 
    length_episode = 300 # default = 300 ! 
    modulo_display = 10
    while True:
        _ +=1
        print_info(_, length_episode, modulo_display)
        action     = np.random.randint(action_size)    # select an action
        env_info   = env.step(action)[brain_name]      # send the action to the environment
        next_state = env_info.vector_observations[0]   # get the next state
        reward     = env_info.rewards[0]               # get the reward
        done       = env_info.local_done[0]            # see if episode has finished
        score     += reward                            # update the score
        episode.append((state, action, reward))
        state      = next_state                        # roll over the state to next time step
        if done:                                        #exit loop if episode finished
            break
    return episode

In [ ]:
def construct_q(env, brain_name, nb_action_unique, num_episodes, generate_episode, gamma=1.0):
    # initialize empty dictionaries of arrays
    import sys
    Nb_de_passage,Gain_total,Q_gain_moyen = get_init(nb_action_unique)
    # loop over episodes
    for i_episode in range(num_episodes):
        print_info(i_episode,num_episodes)# monitor progress
        list_triplet  = generate_episode(env, brain_name)
        rewards       = [triplet[2] for triplet in list_triplet] 
        nb_cartes     = len(rewards)
        discounts     = np.array([gamma**i for i in range(nb_cartes+1)])
        for i , (state, indice_action, reward) in enumerate(list_triplet):
            somme_des_gains_d_ici_a_la_fin        = sum(rewards[i:]*discounts[:-(1+i)])
            tuple_state = tuple(state)
            Nb_de_passage [tuple_state][indice_action] += 1
            Gain_total    [tuple_state][indice_action] += somme_des_gains_d_ici_a_la_fin # Gains entre 1 et -1
            esperated_win                               = Gain_total[tuple_state][indice_action]
            nb_of_passage                               = Nb_de_passage [tuple_state][indice_action]
            gain_moyen                                  = esperated_win / nb_of_passage
            Q_gain_moyen  [tuple_state][indice_action]  = gain_moyen
    """
    Q contains :
        each key = state, on 37 values
        each value = array of 4 values 
            \=> each value correspond to the average return for each of the 4 action
    """
    return Q_gain_moyen

In [ ]:
if False:
    nb_action_unique = 4
    nb_episode       = 10
    Q = construct_q(env,brain_name,nb_action_unique,  nb_episode, generate_episode )

In [ ]:
print("ok")

ok


## 2) controle

In [ ]:
def generate_episode_from_Q(env, Q, epsilon, nA=2):
    import sys
    # settings
    possible_actions = list(range(nA))
    episode   = []
    proba_min = epsilon/nA
    proba_max = 1-epsilon + proba_min
    # code
    print("env going to be reseted")
    state     = env.reset(train_mode=True)
    print("env going reseted")    
    _ = 0
    while True:
        _ += 1
        print_info(_, 100, 10)
        tuple_state = tuple(state)
        # helping the best action to be selected, but still exploring
        probas                         = [proba_min] * nA
        best_action_so_far             = np.argmax(Q[tuple_state])
        probas[best_action_so_far]     = proba_max
        # selecting an action
        action                         = np.random.choice(possible_actions, p=probas)  
        # applying the selected action
        env_info   = env.step(action)[brain_name]      # send the action to the environment
        next_state = env_info.vector_observations[0]   # get the next state
        reward     = env_info.rewards[0]               # get the reward
        done       = env_info.local_done[0]            # see if episode has finished
        
        # saving the result
        episode.append((state, action, reward))
        #going ahead
        state        = next_state
        if done:
            break
    return episode        
        
    

In [ ]:
def update_Q(episode, Q, alpha, gamma):
    # unzipping the data from the episode
    states, actions, rewards = zip(*episode)
    # applying gamma
    discounts = np.array([gamma**i for i in range(len(rewards)+1)])
    
    for i, state in enumerate(states):
        tuple_state = tuple(state)
        # fetching last esperance :
        action_at_step_i           = actions[i]
        previous_expected_value    = Q[tuple_state][action_at_step_i] 
        # comparing with new episode : 
        new_expected_value         = sum(rewards[i:]*discounts[:-(1+i)])
        diff_in_expected_value     = new_expected_value - previous_expected_value
        # updating Q : 
        Q[tuple_state][action_at_step_i] = previous_expected_value + alpha*diff_in_expected_value

    return Q



In [ ]:
def get_policy(Q):
    policy = dict((k,np.argmax(v)) for k, v in Q.items())    
    return policy

In [ ]:
def control(env, nb_of_possible_action, num_episodes, alpha, epsilon, gamma=1.0):
    """
    env : env
    nA : 
    """
    from collections import defaultdict
    import sys
    # initialize empty dictionary of arrays
    Q      = defaultdict(lambda: np.zeros(nb_of_possible_action))
    # loop over episodes
    for i_episode in range(1, num_episodes+1):
        # monitor progress
        print(i_episode); sys.stdout.flush()
        print_info(i_episode, num_episodes, 10)
        ## TODO: complete the function
        print("generate_episode_from_Q")
        episode = generate_episode_from_Q(env, Q,  epsilon, nb_of_possible_action)
        print("update_Q")
        Q       = update_Q(episode, Q, alpha, gamma)
    return get_policy(Q), Q

In [ ]:
try:
    env.close()
except:
    try:
        del env
    except:
        pass
env = UnityEnvironment(file_name="./Banana.app", no_graphics=True)
brain_name = env.brain_names[0]
brain      = env.brains[brain_name]
policy, Q = control(env,4, 3000, 0.03, 0.5)    

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


1
generate_episode_from_Q
env going to be reseted
env going reseted
Episode 300/100.update_Q
2
generate_episode_from_Q
env going to be reseted
env going reseted
Episode 300/100.update_Q
3
generate_episode_from_Q
env going to be reseted
env going reseted
Episode 300/100.update_Q
4
generate_episode_from_Q
env going to be reseted
env going reseted
Episode 300/100.update_Q
5
generate_episode_from_Q
env going to be reseted
env going reseted
Episode 300/100.update_Q
6
generate_episode_from_Q
env going to be reseted
env going reseted
Episode 300/100.update_Q
7
generate_episode_from_Q
env going to be reseted
env going reseted
Episode 300/100.update_Q
8
generate_episode_from_Q
env going to be reseted
env going reseted
Episode 300/100.update_Q
9
generate_episode_from_Q
env going to be reseted
env going reseted
Episode 300/100.update_Q
10
Episode 10/3000.generate_episode_from_Q
env going to be reseted
env going reseted
Episode 300/100.update_Q
11
generate_episode_from_Q
env going to be reseted
en